In [1]:
!pip install -U scikit-learn
!pip install verta
!pip install Minio
!pip install pandas
!pip install alibi
!pip install dill

Requirement already up-to-date: scikit-learn in /Users/faisalmasood/.pyenv/versions/3.7.5/lib/python3.7/site-packages (0.23.2)


In [7]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import joblib
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datetime import datetime

from minio import Minio
from minio.error import ResponseError
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

from verta import Client
import dill


from alibi.explainers import AnchorTabular

# Common Functions - Will be supplied via package
def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient

def record_metrics(classifier, expereiment_name, accuracy_score, hyperparameters):
    client = Client("http://chart-1603715522-webapp:3000")
    proj = client.set_project("HDFC DEmo PRoject")
    client.set_experiment("Iris Classifier")
    run = client.set_experiment_run(expereiment_name)
    
    for key, value in hyperparameters.items():
        run.log_hyperparameters({key : value})
    
    run.log_metric('accuracy', accuracy_score)
    run.log_tags([classifier])

def visualise_iris_data(iris):
    ''' 0 == setosa 
        1 == versicolor
        2 == viginica '''
    #     print(iris)
    
    df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    print(df.describe())
    df["target"] = iris.target
    print(df.head(500))

In [8]:
# Get Experiment ID
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_id = 'IrisScikit' + timestampStr


In [9]:
def train_and_save_model():
#     model = LogisticRegression()
    model = RandomForestClassifier(n_estimators=10)
    
    print('Training model...')
    model.fit(X1, y1)
    print('Model trained!')
    
    accuracy = model.score(X2, y2)
    print("Accuracy: %.2f%%" % (accuracy*100.0))   
#     record_metrics('LogisticREgression', experiment_id, accuracy, {'regularisation': 0.5})
    # record_metrics('RandomForrests', experiment_id, accuracy, {'n_estimators': 10})                                   
    

    filename_p = 'IrisClassifier.sav'
    print('Saving model in %s' % filename_p)
    joblib.dump(model, filename_p)
    print('Model saved!')
    return model

In [10]:
def explain_model(feature_names, model, X_train, X_test_record):
    predict_fn = lambda x: model.predict_proba(x)
    explainer = AnchorTabular(predict_fn, feature_names)
    explainer.fit(X_train, disc_perc=[25, 50, 75])
    explanation = explainer.explain(X_test_record, threshold=0.80)
    print('Anchor: %s' % explanation['anchor'])
    print('Precision: %.2f' % explanation['precision'])
    print('Coverage: %.2f' % explanation['coverage'])
    return explainer

In [13]:
# if __name__ == "__main__":
print('Loading iris data set...')
iris = datasets.load_iris()
feature_names = iris.feature_names
X, y = iris.data, iris.target
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, train_size=0.6)
print('Dataset loaded!')
model = train_and_save_model()
explainer = explain_model(feature_names, model, X1, X2[0])
with open("IrisClassifierExapliner.dill", "wb") as x_f:
    dill.dump(explainer, x_f)

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Accuracy: 93.33%
Saving model in IrisClassifier.sav
Model saved!
Anchor: ['petal width (cm) > 1.90', 'petal length (cm) > 4.15']
Precision: 0.97
Coverage: 0.50


The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.
The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.
The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


In [ ]:
minioClient = get_s3_server()
minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/IrisClassifier.sav' , file_path='./IrisClassifier.sav')